In [ ]:
#import libareries
import math
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from keras.utils import to_categorical
from sklearn.utils.multiclass import unique_labels
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from keras.optimizers import Adam, Adadelta
from keras.layers import LSTM, SimpleRNN, Dense, PReLU, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from sklearn import model_selection
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from keras import metrics
from keras.models import load_model
#from cm import multilabel_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix

In [ ]:
#load dataframes from preproc
x_train = pd.read_pickle('x_Train')
x_test = pd.read_pickle('x_test')
y_test = pd.read_pickle('y_test')
y_train = pd.read_pickle('y_train')
test_data = pd.read_pickle('test_data')

labels = np.array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'])

In [ ]:
def modelToCsv_deeplearning(model):
    """Die Methode schreibt predictions des übergebenen Modells in eine CSV Datei Namens 'submit.csv'.
    Die CSV Datei hat das von Kaggle gewünschte Format."""
    #Droppe Column Id, um die gleichen Features auf den Testdaten wie auf den Trainingsdaten zu haben
    test_daten = test_data.drop(['Id'], axis = 1)
    #füge eine Dimension hinzu, um prediction zu ermöglichen
    test_daten = test_daten.values
    test_daten = test_daten.reshape((test_daten.shape[0], 1, test_daten.shape[1]))
    #Predicte das Ergebnis
    predicted = model.predict_proba(test_daten)

    
    result = pd.DataFrame(predicted, columns = labels)
    result.to_csv('submit_NN.csv',index=True, index_label='Id')

In [ ]:
#Zusätzliche Datapreparation, um es im Neuronalen Netz nutzen zu können
#Nur einmal Ausführen
y_train = to_categorical(y_train)
x_train = x_train.values

y_test = to_categorical(y_test)
x_test = x_test.values

In [ ]:
# Die Erstellung des tiefen neuronalen Netzes
def create_NeuralNet(layer, nodes):
  
    model_nn = Sequential()
    model_nn.add(Dense(nodes, input_shape= x_train[0].shape, init='glorot_uniform', use_bias=True))
    model_nn.add(PReLU())
    model_nn.add(Dropout(0.2))
  
    for i in range(layer):
        model_nn.add(Dense(nodes, init='glorot_uniform', use_bias=True))
        model_nn.add(PReLU())
        model_nn.add(Dropout(0.2))


    model_nn.add(Dense(39, activation='softmax'))


    model_nn.compile(loss='categorical_crossentropy', optimizer=Adadelta(),  metrics=['acc'])
  
    return model_nn

model_nn = create_NeuralNet(30, 39)

In [ ]:
#options
history = model_nn.fit(x_train, 
          y_train,
          validation_data=(x_test, y_test),
          batch_size=512,
          epochs=40)

In [ ]:
scores = model_nn.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model_nn.metrics_names[1], scores[1]*100))

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('NN_Acc.png')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('NN_loss.png')
plt.show()

In [ ]:
y_pred = model_nn.predict(x_test)
np.argmax(y_pred, axis=1) 
pred_Test_NN = (y_pred > 0.5)

In [ ]:
class_names = np.arange(1, 40)

In [ ]:
#Es wurde auf die Confusion Matrix für das Neuronale Netz verzichtet, da die Outputs keine boolean Matrix ist. Zur Richtigstellung muss ein Vektor von Label geschaffen werden, dass aber die CM verfälscht. 

y_pred = model_nn.predict(x_test)
np.argmax(y_pred, axis=1) 
pred_Test_NN = (y_pred > 0.5)

def plot_confusion_matrix(y_test, pred_Test_NN, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_test.argmax(axis=1),pred_Test_NN.argmax(axis=1))
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_test, pred_Test_NN)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    
    fig,ax = plt.subplots(figsize=(20,15))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, pred_Test_NN, classes=class_names,
                      title='Confusion matrix, without normalization')
plt.savefig('NN.png')
plt.show()

In [ ]:
#Zusätzliche Datapreparation, um es im LSTM Netz nutzen zu können
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

In [ ]:
def create_LSTM(layer, nodes):

    model = Sequential()

    model.add(LSTM(nodes,go_backwards= True, use_bias=True, kernel_initializer='glorot_uniform', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(PReLU())
    model.add(Dropout(0.2))
  

    for i in range(layer):
        model.add(LSTM(nodes, use_bias=True, kernel_initializer='glorot_uniform', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
        model.add(PReLU())
        model.add(Dropout(0.2))



    model.add(LSTM(39, use_bias=True, kernel_initializer='glorot_uniform', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=False))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(39, activation='softmax'))


    model.compile(loss='categorical_crossentropy', optimizer=Adadelta(), metrics=['acc'])
  
    return model

model = create_LSTM(50, 70)

In [ ]:
#options
history_lstm = model.fit(x_train, 
          y_train,
          validation_data=(x_test, y_test),
          batch_size=512,
          epochs=2)

In [ ]:
scores_lstm = model.evaluate(x_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores_lstm[1]*100))

In [ ]:
model_nn.save("model_nn.h5")
model.save("model.h5")

In [ ]:
import matplotlib.pyplot as plt


# Plot training & validation accuracy values
plt.plot(history_lstm.history['acc'])
#plt.plot(history.history['val_binary_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('LSTM_Acc.png')
plt.show()

# Plot training & validation loss values
plt.plot(history_lstm.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('LSTM_loss.png')
plt.show()

In [ ]:
# Ausführen, um die Prediciton als CSV (submit.csv) zu speichern
modelToCsv_deeplearning(model_nn)

In [ ]:
# Ausführen, um die Prediciton als CSV (submit.csv) zu speichern
modelToCsv_deeplearning(model)